In [1]:
import torch
import argparse
from model import GPT
from utils import generate
from train import train_model
from utils import getLoaderDataset

In [2]:
N = 64
B = 12
loader, token_dataset = getLoaderDataset(N, B, "./datasets/shakespear_corpus.txt")
print(len(token_dataset))

1003853 64
1003853 64
1003853 64
1003789


In [ ]:
L = 12
d = 768
d_ff = 4 * d
h = 8
V = token_dataset.get_vocab_size()
model = GPT(B, L, d, d_ff, N, h, V)

In [ ]:
cpkt = torch.load("./runs/fat_model/model_6200.pt", map_location=torch.device('cpu'))
model.load_state_dict(cpkt)
print(model)

In [ ]:
new_tokens = generate(model, dataset.encode("Hello World"), 100)
print(dataset.decode(new_tokens))

In [3]:
B = 12
N = 64  # context of up to 256 previous characters
L = 4
h = 4
d = 128
learning_rate = 1e-3
use_lr_decay = False
dataset = './datasets/shakespear_corpus.txt'
out_dir = './runs/'

max_iterations = 1000

class Args(argparse.Namespace):
  batch_size:int = B
  n_tokens:int = N
  n_layers:int = L
  n_heads:int = h
  d_model:int = d
  use_lr_decay:bool = use_lr_decay
  learning_rate: float = learning_rate
  dataset: str = dataset
  max_iterations: int = max_iterations
  out: str = out_dir

In [6]:
model, losses, perplexities = train_model(Args())

1003853 64
1003853 64
111540 64
111540 64
1003853 64
1003853 64
111540 64
111540 64
Epoch: 0, Batch 0, Training Loss: 4.349568843841553, Validation Loss: 3.777230739593506
111540 64
111540 64
Epoch: 0, Batch 100, Training Loss: 2.608867883682251, Validation Loss: 2.5341367721557617
111540 64
111540 64
Epoch: 0, Batch 200, Training Loss: 2.528613328933716, Validation Loss: 2.4632346630096436
111540 64
111540 64
Epoch: 0, Batch 300, Training Loss: 2.4159815311431885, Validation Loss: 2.413639783859253


KeyboardInterrupt: 

In [4]:
model = GPT(Args().batch_size, Args().n_layers, Args().d_model, 3*Args().d_model, Args().n_tokens, Args().n_heads, token_dataset.get_vocab_size())
cpkt = torch.load("./runs/model_500.pt", map_location=torch.device('cpu'))
model.load_state_dict(cpkt)
print(model)

GPT(
  (WTE): Embedding(65, 128)
  (WPE): WPE()
  (blocks): ModuleList(
    (0-3): 4 x Block(
      (CausalSelfAttn): CausalSelfAttention(
        (W_Q): Linear(in_features=128, out_features=128, bias=True)
        (W_K): Linear(in_features=128, out_features=128, bias=True)
        (W_V): Linear(in_features=128, out_features=128, bias=True)
        (W_O): Linear(in_features=128, out_features=128, bias=True)
      )
      (W1): Linear(in_features=128, out_features=384, bias=True)
      (W2): Linear(in_features=384, out_features=128, bias=True)
      (LayerNorm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (LayerNorm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
  )
  (Dropout): Dropout(p=0.2, inplace=False)
  (Final_LayerNorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (LM_Head): Linear(in_features=128, out_features=65, bias=True)
)


In [5]:
new_tokens = generate(model, token_dataset.encode("Hello World"), 200)
print(token_dataset.decode(new_tokens))

/home/gordon/Documents/edu/gordon_ms/Sem3/DeepLearning/Shakespeak/utils.py:115: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_c = torch.tensor(char_id).reshape(shape=(1, 1))


Hello World wedo layy lpprdou, l yother rom ar wis,
Ane t ties, p andin; n, reat mer d theas temend. Tondin,

Wh IUS:
Onis ot mealt met than:
He dire, us o furt st 't mesiny my w, po y s ma se lowatinor, ive t s
